In [1]:
from IPython.display import HTML
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
The raw code for this IPython notebook is by default hidden for easier reading.
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.''')

In [2]:
import pandas
%matplotlib notebook
import general
import discretionary_aua
import vantage_aua
import data_accessing
import combined
import revenue
import costs
import consolidated
import discf
from ipywidgets import widgets
from IPython.display import display
import stats
pandas.set_option('display.float_format', '{:20,.6f}'.format)   #suppressing scientific notation
idx = pandas.IndexSlice

In [3]:
print('Pulling data...')
fund_dic = data_accessing.read_data(data_accessing.fund_data_name,data_accessing.fund_data_types)
nnb_dic = data_accessing.read_data(data_accessing.nnb_data_name,data_accessing.nnb_data_sheet)
discretionary_aua.append_share_class_units(fund_dic) # appending share class units
index_hl_dic = data_accessing.read_data(data_accessing.index_hl_data_name, data_accessing.index_hl_data_sheet)
data_dic = {**fund_dic, **index_hl_dic, **nnb_dic}

input_dic = data_accessing.read_data(data_accessing.assumptions_name, data_accessing.assumptions_sheet)

print('Data pulled successfully.')

Pulling data...
Data pulled successfully.


In [4]:
assumption_button = widgets.Button(description='Update assumptions',layout=widgets.Layout(width='20%',height='40px'))
assumption_button.style.button_color = 'lightgreen'
def update_assumption(b):
    global input_dic
    input_dic = data_accessing.read_data(data_accessing.assumptions_name, data_accessing.assumptions_sheet)
    print ('Assumptions updated successfully...')

assumption_button.on_click(update_assumption)
display(assumption_button)

Button(description='Update assumptions', layout=Layout(height='40px', width='20%'), style=ButtonStyle(button_c…

Assumptions updated successfully...
Assumptions updated successfully...
Assumptions updated successfully...
Assumptions updated successfully...
Assumptions updated successfully...
Assumptions updated successfully...
Assumptions updated successfully...
Assumptions updated successfully...
Assumptions updated successfully...


In [5]:
revenue_types = ['Semi-annual revenue','Annual revenue - FY','Annual revenue - CY','Consolidated semi-annual revenue','Consolidated annual revenue - FY','Consolidated annual revenue - CY']
box_layout = widgets.Layout(display='flex', flex_flow='column', align_items='stretch',border='solid',width='30%')
items = {}
for w in revenue_types:
    items[w] = widgets.Button(description=w,layout=widgets.Layout(width='100%', height='40px'))
    items[w].style.button_color = 'lightgreen'
left_box = widgets.VBox([list(items.values())[0],list(items.values())[1],list(items.values())[2]],layout=box_layout)
right_box = widgets.VBox([list(items.values())[3],list(items.values())[4],list(items.values())[5]],layout=box_layout)
items['Semi-annual revenue'].on_click(lambda x: display(revenue.semi_revenue(data_dic,input_dic)))
items['Annual revenue - FY'].on_click(lambda x: display(revenue.annual_revenue(data_dic,input_dic)))
items['Annual revenue - CY'].on_click(lambda x: display(revenue.annual_revenue(data_dic,input_dic,cal_year=True)))
items['Consolidated semi-annual revenue'].on_click(lambda x: display(consolidated.revenue_analysis(data_dic,input_dic)))
items['Consolidated annual revenue - FY'].on_click(lambda x: display(consolidated.annual_revenue_analysis(data_dic,input_dic)))
items['Consolidated annual revenue - CY'].on_click(lambda x: display(consolidated.annual_revenue_analysis(data_dic,input_dic,cal_year=True)))
widgets.HBox([left_box,right_box])



In [6]:
costs_types = ['Semi-annual costs','Annual costs - FY','Annual costs - CY','Consolidated semi-annual costs','Consolidated annual costs - FY','Consolidated annual costs - CY']
items2 = {}
for w2 in costs_types:
    items2[w2] = widgets.Button(description=w2,layout=widgets.Layout(width='100%', height='40px'))
    items2[w2].style.button_color = 'lightgreen'
left_box2 = widgets.VBox([list(items2.values())[0],list(items2.values())[1],list(items2.values())[2]],layout=box_layout)
right_box2 = widgets.VBox([list(items2.values())[3],list(items2.values())[4],list(items2.values())[5]],layout=box_layout)
items2['Semi-annual costs'].on_click(lambda x: display(costs.semi_costs(input_dic)))
items2['Annual costs - FY'].on_click(lambda x: display(costs.annual_costs(input_dic)))
items2['Annual costs - CY'].on_click(lambda x: display(costs.annual_costs(input_dic,cal_year=True)))
items2['Consolidated semi-annual costs'].on_click(lambda x: display(consolidated.costs_analysis(input_dic)))
items2['Consolidated annual costs - FY'].on_click(lambda x: display(consolidated.annual_costs_analysis(input_dic)))
items2['Consolidated annual costs - CY'].on_click(lambda x: display(consolidated.annual_costs_analysis(input_dic,cal_year=True)))
widgets.HBox([left_box2,right_box2])

In [7]:
ops = [i for i in list(range(general.recent_end_year, general.recent_end_year+10))]
a_slider = widgets.SelectMultiple(options=ops, description='Years for comparison')
display(a_slider)

compare1 = ['NNB comparison','AUA comparison']
box_layout2 = widgets.Layout(display='flex', flex_flow='row', align_items='stretch',border='solid',width='30%')
items3 = {}
for w3 in compare1:
    items3[w3] = widgets.Button(description=w3,layout=widgets.Layout(width='100%', height='40px'))
    items3[w3].style.button_color = 'lightgreen'
box3 = widgets.Box([list(items3.values())[0],list(items3.values())[1]],layout=box_layout2)
items3['NNB comparison'].on_click(lambda x: display(consolidated.get_nnb_compare(data_dic, input_dic,list(a_slider.value))))
items3['AUA comparison'].on_click(lambda x: display(consolidated.get_aua_compare(data_dic, input_dic,list(a_slider.value))))


display(box3)

SelectMultiple(description='Years for comparison', options=(2019, 2020, 2021, 2022, 2023, 2024, 2025, 2026, 20…

Box(children=(Button(description='NNB comparison', layout=Layout(height='40px', width='100%'), style=ButtonSty…

In [8]:
compare2 = ['Interim revenue comparison','Annual revenue comparison', 'Interim costs comparison', 'Annual costs comparison']
box_layout3 = widgets.Layout(display='flex', flex_flow='row', align_items='stretch',border='solid',width='50%')
items4 = {}
for w4 in compare2:
    items4[w4] = widgets.Button(description=w4,layout=widgets.Layout(width='100%', height='40px'))
    items4[w4].style.button_color = 'lightgreen'
top_box4 = widgets.HBox([list(items4.values())[0],list(items4.values())[1]],layout=box_layout3)
bottom_box4 = widgets.HBox([list(items4.values())[2],list(items4.values())[3]],layout=box_layout3)
items4['Interim revenue comparison'].on_click(lambda x: display(consolidated.get_revenue_compare(data_dic, input_dic,True,list(a_slider.value))))
items4['Annual revenue comparison'].on_click(lambda x: display(consolidated.get_revenue_compare(data_dic, input_dic,False,list(a_slider.value))))
items4['Interim costs comparison'].on_click(lambda x: display(consolidated.get_costs_compare(input_dic,True,list(a_slider.value))))
items4['Annual costs comparison'].on_click(lambda x: display(consolidated.get_costs_compare(input_dic,False,list(a_slider.value))))

widgets.VBox([top_box4, bottom_box4])

In [9]:
'''=====current values=======
discounted cash flow period: 9 years
perpetuity growth rate: 0.035
cash flow discounted rate: 0.085
'''
dcf_period = widgets.IntText(value=9,placeholder='Integer',description='DCF period - No. of years:')
display(dcf_period)
perp_growth = widgets.FloatText(value=0.035,placeholder='Percentage as decimals',description='Perpetuity growth rate:')
display(perp_growth)
disct_rate = widgets.FloatText(value=0.085,placeholder='Percentage as decimals',description='Cash flow discount rate:')
display(disct_rate)
current_value = widgets.Checkbox(value=False, description='Fair value now')
display(current_value)
fractional = widgets.Checkbox(value=False, description='Fractional valuation')
display(fractional)

dis_cash = ['Discounted cash flow','Fair value']
items5 = {}
for w5 in dis_cash:
    items5[w5] = widgets.Button(description=w5, layout=widgets.Layout(width='100%', height='40px'))
    items5[w5].style.button_color = 'lightgreen'
fair = widgets.HBox([list(items5.values())[0],list(items5.values())[1]],layout=box_layout3)
items5['Discounted cash flow'].on_click(lambda x: display(discf.disc_cash_flow(data_dic, input_dic,now=current_value.value,fractional=fractional.value,dcf_p=dcf_period.value,disc_rate=disct_rate.value)))
items5['Fair value'].on_click(lambda x: display(discf.fair_value(data_dic, input_dic,now=current_value.value,fractional=fractional.value,dcf_p=dcf_period.value,disc_rate=disct_rate.value,pep_rate=perp_growth.value)))    
display(fair)

IntText(value=9, description='DCF period - No. of years:')

FloatText(value=0.035, description='Perpetuity growth rate:')

FloatText(value=0.085, description='Cash flow discount rate:')

Checkbox(value=False, description='Fair value now')

Checkbox(value=False, description='Fractional valuation')

In [10]:
radio1 = widgets.RadioButtons(options=['month_no','quarter_no','half_no','financial_year','calendar_year'],description='Period',disabled=False)
display(radio1)

others = ['HLF composite monthly return','Total NNB hlf Algo','NNB distribution hlf Algo','Total AUA','% Revenue distribution','Avg AUA distribution', 'Total NNB nnc Algo', 'NNB distribution nnc Algo']
items6 = {}
for w6 in others:
    items6[w6] = widgets.Button(description=w6, layout=widgets.Layout(width='100%', height='40px'))
    items6[w6].style.button_color = 'lightgreen'
other_box1 = widgets.VBox([list(items6.values())[0],list(items6.values())[1],list(items6.values())[2],list(items6.values())[3]],layout=box_layout)
other_box2 = widgets.VBox([list(items6.values())[4],list(items6.values())[5],list(items6.values())[6],list(items6.values())[7]],layout=box_layout)
items6['HLF composite monthly return'].on_click(lambda x: display(discretionary_aua.get_composite_return(data_dic)))
items6['Total NNB hlf Algo'].on_click(lambda x: display(combined.total_nnb(data_dic, input_dic, opt=radio1.value)))
items6['NNB distribution hlf Algo'].on_click(lambda x: display(combined.nnb_distribution(data_dic, input_dic, opt=radio1.value)))
items6['Total AUA'].on_click(lambda x: display(combined.total_aua(data_dic, input_dic)))
items6['% Revenue distribution'].on_click(lambda x: display(stats.summary_revenue_dist_percent(data_dic, input_dic)))
items6['Avg AUA distribution'].on_click(lambda x: display(stats.summary_avg_aua_dist(data_dic, input_dic,radio1.value)))
items6['Total NNB nnc Algo'].on_click(lambda x: display(combined.total_nnb_ci_clientAlgo(data_dic, input_dic, opt=radio1.value)))
items6['NNB distribution nnc Algo'].on_click(lambda x: display(combined.total_nnb_distribution_clientAlgo(data_dic, input_dic, opt=radio1.value)))

widgets.HBox([other_box1, other_box2])

#Financial year based

RadioButtons(description='Period', options=('month_no', 'quarter_no', 'half_no', 'financial_year', 'calendar_y…

In [11]:
nnb_radio = widgets.RadioButtons(options=['day','month','semi-annual','quarter','annual'],description='Period',disabled=False)
nnb_dropdown = widgets.Dropdown(options=['no_select',None],description='Fund option',disabled=False)
unit_dropdown = widgets.Dropdown(options=['acc','inc'],description='Unit type',disabled=False)
display(nnb_radio)
display(nnb_dropdown)
display(unit_dropdown)

nnb_b = widgets.Button(description='Implied HLF nnb period-to-date', layout=widgets.Layout(width='50%', height='40px'))
nnb_b.style.button_color = 'lightgreen'

nnb_b2 = widgets.Button(description='HLF unit change period-to-date', layout=widgets.Layout(width='50%', height='40px'))
nnb_b2.style.button_color = 'lightgreen'
nnb_b.on_click(lambda x: display(stats.hlf_to_date_implied_nnb(data_dic,typ=nnb_radio.value,fund_opt=nnb_dropdown.value)))
nnb_b2.on_click(lambda x: display(stats.hlf_to_date_unit_change(data_dic, unit_type=unit_dropdown.value,typ=nnb_radio.value,fund_opt=nnb_dropdown.value)))

display(nnb_b)
display(nnb_b2)

RadioButtons(description='Period', options=('day', 'month', 'semi-annual', 'quarter', 'annual'), value='day')

Dropdown(description='Fund option', options=('no_select', None), value='no_select')

Dropdown(description='Unit type', options=('acc', 'inc'), value='acc')

Button(description='Implied HLF nnb period-to-date', layout=Layout(height='40px', width='50%'), style=ButtonSt…

Button(description='HLF unit change period-to-date', layout=Layout(height='40px', width='50%'), style=ButtonSt…

In [12]:
cash_radio = widgets.RadioButtons(options=['monthly','quarter_no','half_no','financial_year','calendar_year'],description='Period',disabled=False)
display(cash_radio)


cash_b = widgets.Button(description='Cash margin', layout=widgets.Layout(width='50%', height='40px'))
cash_b.style.button_color = 'lightgreen'
cash_b.on_click(lambda x: display(stats.cash_margin(data_dic, input_dic, cash_radio.value)))
display(cash_b)

RadioButtons(description='Period', options=('monthly', 'quarter_no', 'half_no', 'financial_year', 'calendar_ye…

Button(description='Cash margin', layout=Layout(height='40px', width='50%'), style=ButtonStyle(button_color='l…

In [13]:
eps_radio = widgets.RadioButtons(options=[True, False],description='Calendar year?',disabled=False)
display(eps_radio)

eps_summary = widgets.Button(description='Net earnings and EPS', layout=widgets.Layout(width='50%', height='40px'))
eps_summary.style.button_color = 'lightgreen'
eps_summary.on_click(lambda x: display(stats.summary_total(data_dic, input_dic, cal=eps_radio.value)))
display(eps_summary)


RadioButtons(description='Calendar year?', options=(True, False), value=True)

Button(description='Net earnings and EPS', layout=Layout(height='40px', width='50%'), style=ButtonStyle(button…

In [14]:
hlf_radio = widgets.RadioButtons(options=['month_no','quarter_no','half_no','financial_year','calendar_year'],description='Period',disabled=False)
display(hlf_radio)


buttom3 = ['HLF revenue margin','Monthly revenue', 'HLF avg fund size','Quarter Trading update revenue']
items7 = {}
for w7 in buttom3:
    items7[w7] = widgets.Button(description=w7, layout=widgets.Layout(width='100%', height='40px'))
    items7[w7].style.button_color = 'lightgreen'

top_m = widgets.HBox([list(items7.values())[0],list(items7.values())[1]],layout=box_layout3)
bottom_m = widgets.HBox([list(items7.values())[2],list(items7.values())[3]],layout=box_layout3)

items7['HLF revenue margin'].on_click(lambda x: display(stats.hlf_revenue_margin(data_dic,input_dic,hlf_radio.value)))
items7['Monthly revenue'].on_click(lambda x: display(revenue.monthly_revenue(data_dic, input_dic)))
items7['HLF avg fund size'].on_click(lambda x: display(stats.avg_hlf_size(data_dic, input_dic,hlf_radio.value)))
items7['Quarter Trading update revenue'].on_click(lambda x: display(stats.quarter_revenue(data_dic, input_dic)))


widgets.VBox([top_m, bottom_m])

RadioButtons(description='Period', options=('month_no', 'quarter_no', 'half_no', 'financial_year', 'calendar_y…

In [15]:
display(a_slider)

client_options = ['Total No. of clients','Net new clients','fx effects on funds']
items8 = {}
for w8 in client_options:
    items8[w8] = widgets.Button(description=w8, layout=widgets.Layout(width='100%', height='40px'))
    items8[w8].style.button_color = 'lightgreen'
c_fair = widgets.HBox([list(items8.values())[0],list(items8.values())[1],list(items8.values())[2]],layout=box_layout3)
items8['Total No. of clients'].on_click(lambda x: display(combined.total_client_predt(data_dic, input_dic).loc[list(a_slider.value)]))
items8['Net new clients'].on_click(lambda x: display(combined.net_new_client_predt(data_dic, input_dic).loc[list(a_slider.value)]))
items8['fx effects on funds'].on_click(lambda x: display(vantage_aua.fx_effects_on_funds(data_dic)))
display(c_fair)

SelectMultiple(description='Years for comparison', options=(2019, 2020, 2021, 2022, 2023, 2024, 2025, 2026, 20…

In [16]:
# %load_ext autoreload
# %autoreload 2

In [17]:

#=============================== fair value sensitivity analysis ===========================================

# disc_rate_list = [0.0825,0.085,0.0875,0.09,0.095,0.1]
# pep_rate_list = [0.035,0.0375,0.04,0.0425,0.045]

# df_1y = pandas.DataFrame(columns = disc_rate_list, index=pep_rate_list)
# df_1y.columns.name = 'Cost of Equity'
# df_1y.index.name = 'Long-term Growth Rate'

# for i in pep_rate_list:
#     for j in disc_rate_list:
#         df_1y.loc[i,j] = discf.fair_value(data_dic, input_dic,now=False,fractional=True,dcf_p=9,disc_rate=j,pep_rate=i).loc['Fair value per share','HL']


In [37]:
revenue_20 = revenue.annual_revenue(data_dic, input_dic).sum(axis='columns')[2020]
costs_20 = (costs.annual_costs(input_dic).sum(axis='columns')-costs.annual_costs(input_dic)['capital_expenditure'])[2020]
aua = combined.total_aua(data_dic, input_dic)['total_assets_aua'][6]
pbt = (revenue_20 + costs_20)
eps = pbt * 0.81 / 474318625 * 100

revenue_h2 = revenue.semi_revenue(data_dic, input_dic).sum(axis='columns')[1]
costs_h2 = (costs.semi_costs(input_dic).sum(axis='columns')-costs.semi_costs(input_dic)['capital_expenditure'])[1]
pbt_h2 = (revenue_h2 + costs_h2)
eps_h2 = pbt_h2 * 0.81 / 474318625 * 100

In [38]:
print(str(revenue_h2))
print(str(pbt_h2))
print(str(eps_h2))
print(str(aua/1000000000))
print(str(revenue_20))
print(str(pbt))
print(str(eps))

256760537.52435887
163520537.52435887
27.924611940914335
101.00362646163961
576326209.611314
384118249.61131406
65.5963661948051
